In [1]:
import sqlalchemy as db

In [2]:
engine = db.create_engine('sqlite:///bancofilmes.db', echo=True)

In [3]:
conn = engine.connect()

In [4]:
metadata = db.MetaData()

In [5]:
BDFilmes = db.Table('Filmes', metadata, 
         db.Column('Id', db.Integer(), primary_key= True),
         db.Column('Name', db.String(255), nullable=False),
         db.Column('Rating', db.Float()),
         db.Column('Ano', db.Integer())
         )
metadata.create_all(engine)

2023-05-25 19:44:22,347 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 19:44:22,348 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Filmes")
2023-05-25 19:44:22,349 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 19:44:22,350 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Filmes")
2023-05-25 19:44:22,351 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 19:44:22,352 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Filmes" (
	"Id" INTEGER NOT NULL, 
	"Name" VARCHAR(255) NOT NULL, 
	"Rating" FLOAT, 
	"Ano" INTEGER, 
	PRIMARY KEY ("Id")
)


2023-05-25 19:44:22,352 INFO sqlalchemy.engine.Engine [no key 0.00045s] ()
2023-05-25 19:44:22,758 INFO sqlalchemy.engine.Engine COMMIT


# Fazendo operações no banco

Adicionando uma linha

In [7]:
inserir = db.insert(BDFilmes).values(Id=1, Name='Carandiru', Rating=3, Ano=2010)

In [8]:
resultado = conn.execute(inserir)

2023-05-25 19:52:38,682 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 19:52:38,683 INFO sqlalchemy.engine.Engine INSERT INTO "Filmes" ("Id", "Name", "Rating", "Ano") VALUES (?, ?, ?, ?)
2023-05-25 19:52:38,683 INFO sqlalchemy.engine.Engine [generated in 0.00098s] (1, 'Carandiru', 3.0, 2010)


In [9]:
output = conn.execute(BDFilmes.select()).fetchall()
print(output)

2023-05-25 19:55:13,155 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes"
2023-05-25 19:55:13,156 INFO sqlalchemy.engine.Engine [generated in 0.00046s] ()
[(1, 'Carandiru', 3.0, 2010)]


Adicionando várias linhas:

In [10]:
query = db.insert(BDFilmes)
valores = [{'Id':2, 'Name':'Bacarau', 'Rating':4.5, 'Ano': 2021},
            {'Id':3, 'Name':'As branquelas', 'Rating':2.5, 'Ano': 2004},

]

resultado = conn.execute(query, valores)

2023-05-25 19:58:34,862 INFO sqlalchemy.engine.Engine INSERT INTO "Filmes" ("Id", "Name", "Rating", "Ano") VALUES (?, ?, ?, ?)
2023-05-25 19:58:34,865 INFO sqlalchemy.engine.Engine [generated in 0.00181s] [(2, 'Bacarau', 4.5, 2021), (3, 'As branquelas', 2.5, 2004)]


In [11]:
output = conn.execute(BDFilmes.select()).fetchall()
print(output)

2023-05-25 19:59:00,263 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes"
2023-05-25 19:59:00,263 INFO sqlalchemy.engine.Engine [cached since 227.1s ago] ()
[(1, 'Carandiru', 3.0, 2010), (2, 'Bacarau', 4.5, 2021), (3, 'As branquelas', 2.5, 2004)]


# Consultas

In [56]:
output = conn.execute('SELECT * FROM Filmes')
print(output.fetchall())

ObjectNotExecutableError: Not an executable object: 'SELECT * FROM Filmes'

In [53]:
query = BDFilmes.select().where(db.and_(BDFilmes.columns.Ano == '2004', BDFilmes.columns.Rating <=4))
output = conn.execute(query)
print(output.fetchall())

2023-05-25 23:04:23,103 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes" 
WHERE "Filmes"."Ano" = ? AND "Filmes"."Rating" <= ?
2023-05-25 23:04:23,104 INFO sqlalchemy.engine.Engine [cached since 23.85s ago] ('2004', 4)


OperationalError: (sqlite3.OperationalError) no such table: Filmes
[SQL: SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes" 
WHERE "Filmes"."Ano" = ? AND "Filmes"."Rating" <= ?]
[parameters: ('2004', 4)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Integrando com o Pandas

In [21]:
import pandas as pd

In [22]:
query = BDFilmes.select().where(BDFilmes.columns.Ano.in_([2003,2004]))
output = conn.execute(query)
results = output.fetchall()
results

2023-05-25 20:11:22,088 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes" 
WHERE "Filmes"."Ano" IN (?, ?)
2023-05-25 20:11:22,089 INFO sqlalchemy.engine.Engine [generated in 0.00257s] (2003, 2004)


[(3, 'As branquelas', 2.5, 2004)]

In [25]:
dados = pd.DataFrame(results)
dados

,Id,Name,Rating,Ano
0,3,As branquelas,2.5,2004


In [29]:
dados.to_csv('bancofilmesv2.csv', index=False, sep=';')

Como converter um dataframe para uma tabela SQL

In [30]:
df = pd.read_csv('bancofilmesv2.csv', sep=';')
df

,Id,Name,Rating,Ano
0,3,As branquelas,2.5,2004


In [39]:
df.to_sql(con=engine, name='filmesv2', if_exists='replace', index=False)

2023-05-25 20:24:50,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 20:24:50,153 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("filmesv2")
2023-05-25 20:24:50,154 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:24:50,155 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("filmesv2")
2023-05-25 20:24:50,155 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:24:50,155 INFO sqlalchemy.engine.Engine ROLLBACK
2023-05-25 20:24:50,157 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 20:24:50,157 INFO sqlalchemy.engine.Engine 
CREATE TABLE filmesv2 (
	"Id" BIGINT, 
	"Name" TEXT, 
	"Rating" FLOAT, 
	"Ano" BIGINT
)


2023-05-25 20:24:50,158 INFO sqlalchemy.engine.Engine [no key 0.00042s] ()
2023-05-25 20:24:55,687 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) database is locked
[SQL: 
CREATE TABLE filmesv2 (
	"Id" BIGINT, 
	"Name" TEXT, 
	"Rating" FLOAT, 
	"Ano" BIGINT
)

]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [33]:
df = pd.read_csv('bancofilmesv2.csv', sep=';')

In [38]:
conn2 = engine.connect()
metadata2 = db.MetaData()
bdfilmes2 = db.Table('filmesv2', metadata2, autoload=True, autoload_with=engine)

TypeError: Additional arguments should be named <dialectname>_<argument>, got 'autoload'

In [ ]:
query = bdfilmes2.select()
exe = conn.execute(query)
result = exe.fetchmany(5)
for r in result:
    print(r)

# Outro formato de criação de bancos

In [40]:
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base

In [41]:
engine = db.create_engine("sqlite:///..//impacta.db", echo=True)

In [42]:
conn = engine.connect()

In [43]:
Base = declarative_base()

class School(Base):
    __tablename__ = 'impacta'

    id = Column(Integer, primary_key = True)
    name = Column(String)

    def __init__(self, name):
        self.name = name

    Base.metadata.create_all(engine)

2023-05-25 20:32:06,312 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 20:32:06,314 INFO sqlalchemy.engine.Engine COMMIT


C:\Users\andre\AppData\Local\Temp\ipykernel_17760\2641082970.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
